In [1]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
pyo.init_notebook_mode(connected=True)

pd.options.plotting.backend = 'plotly'
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas_ta as pandas_ta
%matplotlib inline


/Users/mac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from pyspark.sql.functions import array, create_map, struct ,lit , explode,col,expr,udf,row_number
from pyspark.sql import functions as F,Window
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.session import SparkSession
from pyspark.sql import SparkSession   
  
spark=SparkSession.builder.appName("test").master("local[*]").getOrCreate()      

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 20:48:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = "stock_prices_data.csv"
pysparkdf = spark.read.option("header",True).csv(path)
pysparkdf.show(3)

+----------+------------------+-----------------+------------------+------------------+------------------+---------+
|      Date|              Open|             High|               Low|             Close|         Adj Close|   Volume|
+----------+------------------+-----------------+------------------+------------------+------------------+---------+
|2021-12-01|167.47999572753906|170.3000030517578|164.52999877929688|164.77000427246094|162.70103454589844|152052500|
|2021-12-02|158.74000549316406|164.1999969482422| 157.8000030517578|163.75999450683594|161.70372009277344|136739200|
|2021-12-03|164.02000427246094|164.9600067138672|159.72000122070312|161.83999633789062|159.80783081054688|118023100|
+----------+------------------+-----------------+------------------+------------------+------------------+---------+
only showing top 3 rows



In [4]:
# above for candlestick

In [5]:
def patterns(data):
  high = data['High']
  low = data['Low']
  open = data['Open']
  close= data['Close']

  data['1'] = talib.CDL2CROWS(open, high, low, close)
  data['2'] = talib.CDL3BLACKCROWS(open, high, low, close)
  data['3'] = talib.CDL3INSIDE(open, high, low, close)
  data['4'] = talib.CDL3LINESTRIKE(open, high, low, close)
  data['5'] = talib.CDL3OUTSIDE(open, high, low, close)
  data['6'] = talib.CDL3STARSINSOUTH(open, high, low, close)
  data['7'] = talib.CDL3WHITESOLDIERS(open, high, low, close)
  data['8'] = talib.CDLABANDONEDBABY(open, high, low, close, penetration=0)
  data['9'] = talib.CDLADVANCEBLOCK(open, high, low, close)
  data['10'] = talib.CDLBELTHOLD(open, high, low, close)
  data['11'] = talib.CDLBREAKAWAY(open, high, low, close)
  data['12'] = talib.CDLCLOSINGMARUBOZU(open, high, low, close)
  data['13'] = talib.CDLCONCEALBABYSWALL(open, high, low, close)
  data['14'] = talib.CDLCOUNTERATTACK(open, high, low, close)
  data['15'] = talib.CDLDARKCLOUDCOVER(open, high, low, close, penetration=0)
  data['16'] = talib.CDLDOJI(open, high, low, close)
  data['17'] = talib.CDLDOJISTAR(open, high, low, close)
  data['18'] = talib.CDLDRAGONFLYDOJI(open, high, low, close)
  data['19'] = talib.CDLENGULFING(open, high, low, close)
  data['20'] = talib.CDLEVENINGDOJISTAR(open, high, low, close, penetration=0)
  data['21'] = talib.CDLEVENINGSTAR(open, high, low, close, penetration=0)
  data['22'] = talib.CDLGAPSIDESIDEWHITE(open, high, low, close)
  data['23'] = talib.CDLGRAVESTONEDOJI(open, high, low, close)
  data['24'] = talib.CDLHAMMER(open, high, low, close)
  data['25'] = talib.CDLHANGINGMAN(open, high, low, close)
  data['26'] = talib.CDLHARAMI(open, high, low, close)
  data['27'] = talib.CDLHARAMICROSS(open, high, low, close)
  data['28'] = talib.CDLHIGHWAVE(open, high, low, close)
  data['29'] = talib.CDLHIKKAKE(open, high, low, close)
  data['30'] = talib.CDLHIKKAKEMOD(open, high, low, close)
  data['31'] = talib.CDLHOMINGPIGEON(open, high, low, close)
  data['32'] = talib.CDLIDENTICAL3CROWS(open, high, low, close)
  data['33'] = talib.CDLINNECK(open, high, low, close)
  data['34'] = talib.CDLINVERTEDHAMMER(open, high, low, close)
  data['35'] = talib.CDLKICKING(open, high, low, close)
  data['36'] = talib.CDLKICKINGBYLENGTH(open, high, low, close)
  data['37'] = talib.CDLLADDERBOTTOM(open, high, low, close)
  data['38'] = talib.CDLLONGLEGGEDDOJI(open, high, low, close)
  data['39'] = talib.CDLLONGLINE(open, high, low, close)
  data['40'] = talib.CDLMARUBOZU(open, high, low, close)
  data['41'] = talib.CDLMATCHINGLOW(open, high, low, close)
  data['42'] = talib.CDLMATHOLD(open, high, low, close, penetration=0)
  data['43'] = talib.CDLMORNINGDOJISTAR(open, high, low, close, penetration=0)
  data['44'] = talib.CDLMORNINGSTAR(open, high, low, close, penetration=0)
  data['45'] = talib.CDLONNECK(open, high, low, close)
  data['46'] = talib.CDLPIERCING(open, high, low, close)
  data['47'] = talib.CDLRICKSHAWMAN(open, high, low, close)
  data['48'] = talib.CDLRISEFALL3METHODS(open, high, low, close)
  data['49'] = talib.CDLSEPARATINGLINES(open, high, low, close)
  data['50'] = talib.CDLSHOOTINGSTAR(open, high, low, close)
  data['51'] = talib.CDLSHORTLINE(open, high, low, close)
  data['52'] = talib.CDLSPINNINGTOP(open, high, low, close)
  data['53'] = talib.CDLSTALLEDPATTERN(open, high, low, close)
  data['54'] = talib.CDLSTICKSANDWICH(open, high, low, close)
  data['55'] = talib.CDLTAKURI(open, high, low, close)
  data['56'] = talib.CDLTASUKIGAP(open, high, low, close)
  data['57'] = talib.CDLTHRUSTING(open, high, low, close)
  data['58'] = talib.CDLTRISTAR(open, high, low, close)
  data['59'] = talib.CDLUNIQUE3RIVER(open, high, low, close)
  data['60'] = talib.CDLUPSIDEGAP2CROWS(open, high, low, close)
  data['61'] = talib.CDLXSIDEGAP3METHODS(open, high, low, close)
  data['pattern_score'] = data.loc[:, '1':'61'].sum(axis=1)

  return data

In [6]:
def StochRSI(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()

    return stochrsi_K, stochrsi_D
  
def StochRSI_EMA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.ewm(span=smoothK).mean()
    stochrsi_D = stochrsi_K.ewm(span=smoothD).mean()

    return stochrsi_K, stochrsi_D

In [7]:
def get_indicators(data):
    # Get MACD
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])
    # ema10 works well for crypto
    # data['macd_signal'] = data['macd_signal'].ewm(span=10).mean()
    data['macd_EMA50'] = data['macd'].ewm(span=50).mean()
    # data['macd_hist'] = data['macd_hist'].ewm(span=10).mean()
    data['macd_signal_s1'] = data['macd_signal'].shift(1)
    data['macd_s1'] = data['macd'].shift(1)
    data['macd_hist_s1'] = data['macd_hist'].shift(1)
    data['macd_hist_s2'] = data['macd_hist'].shift(2)
    data['macd_hist_s3'] = data['macd_hist'].shift(3)
    
    # Get EMAs and MAs
    data['EMA9'] = data['Close'].ewm(span=9).mean()
    data['EMA9_s1'] = data['EMA9'].shift(1)
    data['EMA5'] = data['Close'].ewm(span=5).mean()
    data['EMA10'] = data['Close'].ewm(span=10).mean()
    data['EMA50'] = data['Close'].ewm(span=50).mean()
    data['EMA50_s1'] = data['EMA50'].shift(1)
    data['EMA200'] = data['Close'].ewm(span=200).mean()
    data['EMA300'] = data['Close'].ewm(span=300).mean()

    data['MA200'] = data['Close'].rolling(window=200).mean()
    data['MA5'] = data['Close'].rolling(window=5).mean()
    data['MA5_s1'] = data['MA5'].shift(1)
    data['MA10'] = data['Close'].rolling(window=10).mean()
    data['MA300'] = data['Close'].rolling(window=300).mean()
    data['MA10_s1'] = data['MA10'].shift(1)

    data['MA50'] = data['Close'].rolling(window=50).mean()

    # CCI 
    data["CCI"] = talib.CCI(data['High'], data['Low'], data['Close'], timeperiod=20)
    data["CCI_s1"] = data["CCI"].shift(1)
    
    # # Money flow index 14
    data['Money_flow_index14'] = ta.volume.money_flow_index(data['High'], data['Low'], data['Close'],
                                                          data['Volume'] , fillna =False)
    # Mean centered money flow index
    data['Mean_Center_MFI'] = data['Money_flow_index14'] - data['Money_flow_index14'].rolling(200, min_periods=20).mean()
    data['EMA50_Mean_Center_MFI'] = data['Mean_Center_MFI'].ewm(span=50).mean()

    # BB
    data["upperband"], data["middleband"], data["lowerband"] = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

    ## FULL STO
    data['stochrsi_K'], data['stochrsi_D'] = StochRSI_EMA(data['Close'], period=14, smoothK=3, smoothD=3)
    data['stochrsi_K_s1'] = data['stochrsi_K'].shift(1)
    data['stochrsi_D_s1'] = data['stochrsi_D'].shift(1)
    data['stochrsi_K_EMA50'] = data['stochrsi_K'].ewm(span=50).mean()
    data['stochrsi_K_EMA50_s1'] = data['stochrsi_K_EMA50'].shift(1)

    # Volume
    data['Volume_s1'] = data['Volume'].shift(1)
    data['Volume_pct'] = data['Volume'].apply(np.log).pct_change().ewm(span=50).mean() # percentage change
    data['Volume_pct_s1'] = data['Volume_pct'].shift(1) 

    # Volume weighted average price
    data['VWAP'] = np.cumsum(data['Volume']*(data['High']+data['Low'])/2) / np.cumsum(data['Volume'])
    data['VWAP_s1'] = data['VWAP'].shift(1)

    # Previous Candle
    data['High_s1'] = data['High'].shift(1)
    data['Low_s1'] = data['Low'].shift(1)
    data['Open_s1'] = data['Open'].shift(1)
    data['Close_s1'] = data['Close'].shift(1)

    data['High_s2'] = data['High'].shift(2)
    data['Low_s2'] = data['Low'].shift(2)
    data['Open_s2'] = data['Open'].shift(2)
    data['Close_s2'] = data['Close'].shift(2)

    return data

In [8]:

def buy_conditions(s):

    # Pattern Score
    # if (s['19'] >= 100) and (s['Close'] < s['MA300']):
    #   return s['Close']
    # if (s['24'] >= 100) and (s['Close'] < s['MA300']):
    #   return s['Close']
    # if (s['48'] >= 100):
    #   return s['Close']
    # if (s['46'] >= 100) and (s['Close'] < s['MA300']):
    #   return s['Close']

    # MACD
    if (s['macd_signal_s1'] > s['macd_s1']) and (s['macd'] > s['macd_signal']): 
      return s['Close']
    if (s['macd_hist'] < 0) and (s['macd_hist_s3'] < s['macd_hist_s2'] < s['macd_hist_s1'] <s['macd_hist']) and (s['Close'] < s['MA200']): 
      return s['Close']

    # if (s['macd_hist'] < 0) and (s['macd_hist_s3'] < s['macd_hist_s2'] < s['macd_hist_s1'] <s['macd_hist']) and \
    # (s['Close'] > s['MA200']) and (s['Open'] < s['MA5']): 
    #   return s['Close']
    
    # # STOCH RSI
    # if (s['stochrsi_K_EMA50_s1'] > s['stochrsi_K_s1']) and (s['stochrsi_K_EMA50'] < s['stochrsi_K']) and (s['stochrsi_K_EMA50'] < 0.3): 
    #         return s['Close']

    # # EMA
    # if (s['EMA9_s1'] < s['EMA50_s1']) and (s['EMA9'] > s['EMA50']) and (s['Close'] < s['MA200']): 
    #   return s['Close']

    # # CCI
    # if (s['Close'] < s['MA200']):
    #   if (s['CCI_s1'] < 0) and (s['CCI'] > 0): 
    #     return s['Close']
    
    # if (s['CCI'] < -250) and (s['Close'] < s['MA200']): 
    #     return s['Close']
    else:
        return None # Do Nothing


def sell_conditions(s):

    # Pattern Score
    # if (s['26'] <= -100) and (s['Close'] > s['MA300']):
    #   return s['Close']
    # if (s['15'] <= -100) and (s['Close'] > s['MA300']):
    #   return s['Close']
    # if (s['21'] <= -100) and (s['Close'] > s['MA200']):
    #   return s['Close']
    # if (s['25'] <= -100) and (s['Close'] > s['MA300']):
    #   return s['Close']


    # if (s['MA5_s1'] > s['MA10_s1']) and (s['MA5'] < s['MA10']) and (s['MA200'] < s['Close'] ): 
    #   return s['Close']

    # MACD
    if (s['macd_signal_s1'] < s['macd_s1']) and (s['macd'] < s['macd_signal']) and (s['Close'] > s['MA200']): 
          return s['Close']
    if (s['macd_hist'] > 0) and (s['macd_hist_s3'] > s['macd_hist_s2'] > s['macd_hist_s1'] > s['macd_hist']) and (s['Close'] > s['MA200']): 
      return s['Close']

    if (s['macd_hist'] > 0) and (s['macd_hist_s3'] > s['macd_hist_s2'] > s['macd_hist_s1'] > s['macd_hist']) and \
    (s['Close'] < s['MA200']) and (s['middleband'] > s['Close']): 
      return s['Close']

    # # STOCH RSI
    # if (s['stochrsi_K_EMA50_s1'] < s['stochrsi_K_s1']) and (s['stochrsi_K_EMA50'] > s['stochrsi_K'])  and (s['stochrsi_K_EMA50'] > 0.7): 
    #         return s['Close']

    # # EMA
    # if (s['Open'] > s['upperband']) and (s['Close'] > s['upperband']) and (s['Close'] > s['EMA50'] ):
    #   return s['Close']
    # if (s['EMA9_s1'] > s['EMA50_s1']) and (s['EMA9'] < s['EMA50']) and (s['Close'] > s['MA200']): 
    #   return s['Close']

    # # CCI
    # if (s['CCI_s1'] > 0) and (s['CCI'] < 0) and (s['macd_signal'] > s['macd']) and (s['Close'] > s['MA200']): 
    #     return s['Close']

    # if (s['CCI'] > 200) and (s['Close'] > s['MA200']): 
    #     return s['Close']

    else:
        return None # Do Nothing


def macd_buy_signal(s):
    if (s['macd_signal_s1'] > s['macd_s1']) and (s['macd'] > s['macd_signal']): 
      return s['macd']
    # if (s['macd_hist'] < 0) and (s['macd_hist_s3'] < s['macd_hist_s2'] < s['macd_hist_s1'] <s['macd_hist']) and \
    # (s['Close'] < s['MA200']):  # add dont buy if greater than upppband
    #   return s['macd']
    else:
        return None # Do Nothing

def macd_sell_signal(s):
    if (s['macd_signal_s1'] < s['macd_s1']) and (s['macd'] < s['macd_signal']): 
      return s['macd']
    # if (s['macd_hist'] > 0) and (s['macd_hist_s3'] > s['macd_hist_s2'] > s['macd_hist_s1'] > s['macd_hist']) and \
    #  (s['EMA5'] > s['EMA10']): 
    #   return s['macd']
    else:
        return None # Do Nothing

# def cci_buy_signal(s):
#     if (s['CCI_s1'] < 0) and (s['CCI'] > 0): 
#       return s['CCI']
#     else:
#         return None # Do Nothing

# def cci_sell_signal(s):
#     if (s['CCI_s1'] > 0) and (s['CCI'] < 0): 
#       return s['CCI']
#     else:
#         return None # Do Nothing

def rsi_buy_signal(s):
    if (s['stochrsi_K_EMA50_s1'] > s['stochrsi_K_s1']) and (s['stochrsi_K_EMA50'] < s['stochrsi_K']): 
      return s['stochrsi_K_EMA50']*100
    else:
        return None # Do Nothing

def rsi_sell_signal(s):
    if (s['stochrsi_K_EMA50_s1'] < s['stochrsi_K_s1']) and (s['stochrsi_K_EMA50'] > s['stochrsi_K']): 
      return s['stochrsi_K_EMA50']*100
    else:
        return None # Do Nothing

# def vwap_buy_signal(s):
#     # if (s['EMA9_s1'] < s['VWAP_s1']) and (s['EMA9'] > s['VWAP']): 
#     #   return s['EMA9']
#     if (s['EMA9_s1'] < s['EMA50_s1']) and (s['EMA9'] > s['EMA50']) and (s['Close'] < s['MA200']): 
#       return s['EMA9']

#     else:
#         return None # Do Nothing

# def vwap_sell_signal(s):
#     # if (s['EMA9_s1'] > s['VWAP_s1']) and (s['EMA9'] < s['VWAP']): 
#     #   return s['EMA9']
#     if (s['EMA9_s1'] > s['EMA50_s1']) and (s['EMA9'] < s['EMA50']) and (s['Close'] > s['MA200']): 
#       return s['EMA9']
#     else:
#         return None # Do Nothing

# CHANGE TO MFI AND BUY AND SELL AT EMA50 TOUCHING -20 20
# turn this into when ema50 crosses the mfi above -+19
def MFI_buy_signal(s):
    if (s['EMA50_Mean_Center_MFI'] < -20) : 
      return s['EMA50_Mean_Center_MFI']
    else:
        return None # Do Nothing

def MFI_sell_signal(s):
    if (s['EMA50_Mean_Center_MFI'] > 20): 
      return s['EMA50_Mean_Center_MFI']
    else:
        return None # Do Nothing

In [10]:
import talib

tick = ['AAPL']
df = yf.download(tick, start='2021-12-01', end='2023-01-01')
df.to_csv("stock_prices_data.csv")

[*********************100%%**********************]  1 of 1 completed


In [11]:
def trade_env(data, tick,buy_price=20,sell_price=10,discount=1/2,CAPITAL=100,print_ = True):
    CAPITAL = CAPITAL # units
    COMMISSION = 0.01
    units = 0
    Buy_Signal = data.Buy_Signal.tolist()
    middleband = data.middleband.tolist()
    Sell_Signal = data.Sell_Signal.tolist()
    Close = data.Close.tolist()
    index = np.arange(len(data))
    def neat_date(x):
        return str(x).replace("00:00:00","")
    
    for i in range(len(Buy_Signal)):
    
        if (Buy_Signal[i] > 0) and (CAPITAL > 0) and (Buy_Signal[i] < middleband[i]): # Buy 20 when under the middleband 
          units = units + (buy_price/Buy_Signal[i])*(1-COMMISSION) # How much to buy - commisson
          CAPITAL = CAPITAL - buy_price
          # if (print_ == True):
          #   print(f'{neat_date(index[i])} : BUY {20/Buy_Signal[i]} at {Buy_Signal[i]}')
        
        if (Buy_Signal[i] > 0) and (CAPITAL > 0) and (Buy_Signal[i] > middleband[i]): # Buy 10 when over the middleband 
          units = units + (int(buy_price*discount)/Buy_Signal[i])*(1-COMMISSION) # How much to buy - commisson
          CAPITAL = CAPITAL - int(buy_price*discount)
          # if (print_ == True):
          #   print(f'{neat_date(index[i])} : BUY {10/Buy_Signal[i]} at {Buy_Signal[i]}')
        
        
        if (units > 0):
          if (Sell_Signal[i] > 0): # Sell
            sell_units = sell_price/Sell_Signal[i]
            CAPITAL = CAPITAL + ((Sell_Signal[i]*(sell_units))*(1-COMMISSION)) # sellign 50 percent everytime - commisson
            # if (print_ == True):
            #   print(f'{neat_date(index[i])} : SELL {units} at {Sell_Signal[i]}')
            units = units-sell_units
        
        if (i == (data.shape[0]-1)): # Print portfolio value of last day
          portfolio_val = (units*Close[i]) + CAPITAL # Current portfolio value
          #print(f'{neat_date(index[i])} : APPROX {tick} PORTFOLIO VALUE {portfolio_val}')
        
    return portfolio_val,CAPITAL,units

In [15]:
data = get_indicators(df)
data = patterns(data)

data['macd_buy_signal'] = data.apply(macd_buy_signal, axis=1)
data['macd_sell_signal'] = data.apply(macd_sell_signal, axis=1)
# data['CCI_buy_signal'] = data.apply(cci_buy_signal, axis=1)
# data['CCI_sell_signal'] = data.apply(cci_sell_signal, axis=1)
data['MFI_buy_signal'] = data.apply(MFI_buy_signal, axis=1)
data['MFI_sell_signal'] = data.apply(MFI_sell_signal, axis=1)
# data['VWAP_buy_signal'] = data.apply(vwap_buy_signal, axis=1)
# data['VWAP_sell_signal'] = data.apply(vwap_sell_signal, axis=1)
data['RSI_buy_signal'] = data.apply(rsi_buy_signal, axis=1)
data['RSI_sell_signal'] = data.apply(rsi_sell_signal, axis=1)
data = patterns(data)

data['Buy_Signal'] = data.apply(buy_conditions, axis=1)
data['Sell_Signal'] = data.apply(sell_conditions, axis=1)


In [18]:
CAPITAL = 10000
buy_prices = np.arange(1,100).tolist()
sell_prices = np.arange(1,100).tolist()
accounts = np.arange(0.01,20).tolist()

best_invest = -float('inf')

for i in buy_prices:
    for j in sell_prices:
        for k in accounts:
            portval = []
            val,capital,unit = trade_env(data,tick,i,j,k,CAPITAL)

            if(val > best_invest):
                best_invest = val
                print(f'best : {val}, when buy price is {i}, sell price is {j}, account price is {k}')
                complex = []
                complex.append(val)
                complex.append(i)
                complex.append(j)
                complex.append(k)               
                portval.append(complex)

best : 9999.682965339722, when buy price is 1, sell price is 10, account price is 0.01
best : 9999.737409549776, when buy price is 1, sell price is 11, account price is 0.01
best : 9999.79185375983, when buy price is 1, sell price is 12, account price is 0.01
best : 9999.846297969887, when buy price is 1, sell price is 13, account price is 0.01
best : 9999.90074217994, when buy price is 1, sell price is 14, account price is 0.01
best : 9999.955186389994, when buy price is 1, sell price is 15, account price is 0.01
best : 10000.009630600049, when buy price is 1, sell price is 16, account price is 0.01
best : 10000.064074810103, when buy price is 1, sell price is 17, account price is 0.01
best : 10000.118519020158, when buy price is 1, sell price is 18, account price is 0.01
best : 10000.172963230212, when buy price is 1, sell price is 19, account price is 0.01
best : 10000.227407440267, when buy price is 1, sell price is 20, account price is 0.01
best : 10000.281851650321, when buy pric

In [14]:
a

NameError: name 'a' is not defined

In [ ]:
capital

In [ ]:

df['MA50'] = df['Close'].rolling(window=50, min_periods=0).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=0).mean()

df['rsi'] = pandas_ta.rsi(close=df['Close'], length=20)

def compute_macd(data):
    data ['ema12'] = data ['Adj Close'].ewm(span=12). mean ()
    data ['ema26'] = data ['Adj Close'].ewm(span=26).mean ( )
    data ['macd'] = data ['ema12'] - data ['ema26']
    data ['macd signal'] = data ['macd'].ewm(span=9). mean ( )
    data ['macd_histogram'] = data ['macd'] - data ['macd signal']
    return data

compute_macd(df)

df['atr'] = ta.volatility.AverageTrueRange(
    df.High,
    df.Low,
    df.Close,
    window=2,
    fillna=False
).average_true_range()
df = df.reset_index()
df.head(3)

In [ ]:
# Create two charts on the same figure.
ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
ax2 = plt.subplot2grid((10,1), (4,0), rowspan = 2, colspan = 1)
ax3 = plt.subplot2grid((10,1), (6,0), rowspan = 2, colspan = 1)

# First chart:
# Plot the closing price on the first chart
ax1.plot(df['Close'], linewidth=2)


ax1.set_title('apple Close Price')

# Second chart
# Plot the RSI
ax2.plot(df['rsi'], color='orange', linewidth=1)
# Add two horizontal lines, signalling the buy and sell ranges.
# Oversold
ax2.axhline(30, linestyle='--', linewidth=1.5, color='green')
# Overbought
ax2.axhline(70, linestyle='--', linewidth=1.5, color='red')


ax3.plot(df['macd'], label='MACD Line', color='blue')
ax3.plot(df['macd signal'], label='Signal Line', color='red')

y = df['macd_histogram'].tolist()

color = [[1,0,0] if y[i] < 0 and y[i] < y[i-1] else ([1,0,1] if y[i] < 0 and y[i] > y[i-1]  
         else ([0,0,1] if y[i] > 0 and y[i] > y[i-1] else [0,1,1]))   for i in range(len(y))]

ax3.bar(df.index, y, label='Histogram', color=color, alpha=0.5)



In [ ]:
# Set the theme of our chart
plt.style.use('fivethirtyeight')

# Make our resulting figure much bigger
plt.rcParams['figure.figsize'] = (20, 20)

In [ ]:
df['garman_klass_vol'] = ((np.log(df['High'])-np.log(df['Low']))**2)/2-(2*np.log(2)-1)*((np.log(df['Adj Close'])-np.log(df['Open']))**2)


df['rsi'] = df.groupby(['Adj Close']).transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])


df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['High'],
                        low=stock_data['Low'],
                        close=stock_data['Close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

df['dollar_volume'] = (df['Adj Close']*df['Volume'])/1e6

df

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show(renderer="svg")

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
               vertical_spacing=0.10, subplot_titles=('CBA', 'Volume'),
               row_width=[0.2, 0.7])

fig.add_trace(go.Candlestick(x=df.index, open=df["Open"], high=df["High"],
                low=df["Low"], close=df["Close"], name="OHLC"),
                row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df["MA50"], marker_color='grey',name="MA50"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["MA200"], marker_color='lightgrey',name="MA200"), row=1, col=1)

fig.add_trace(go.Bar(x=df.index, y=df['Volume'], marker_color='red', showlegend=False), row=2, col=1)

fig.update_layout(
    title='CBA historical price chart',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Price ($/share)',
        titlefont_size=14,
        tickfont_size=12,
    ),
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    paper_bgcolor='LightSteelBlue'
)

fig.update(layout_xaxis_rangeslider_visible=True)
fig.show(renderer="colab")


In [ ]:
path = "stock_prices_data.csv"
df = spark.read.option("header",True).csv(path)
df.show()

In [ ]:
from pyspark.sql.functions import col
df =df.withColumnRenamed('Open', 'open')
df.show()

In [ ]:
df2 = spark.read.option("header",True) \
     .csv(path)
df2.show()